 # Opis notatnika
 Głównym celem w tym notatniku jest  odpowiednie dostosowanie struktury danych z plików źródłowych do formatu zgodnego z `Postgres`, a następnie wgranie ich na serwer.

## Połączenie z bazą danych
Konfiguracja połączenia utwórz zmienne:
- `username` - nazwa użytkownika bazy,
- `password` - hasło do bazy,
- `host` - adres naszej bazy danych, jeśli baza jest postawiona na naszej maszynie wtedy będzie to po prostu `localhost`,
- `database` - nazwa bazy danych np. `postgresql`
- `port` - domyślnie `5432`

In [1]:
username = ''
password = ''

host = 'localhost'
database = 'airlines'
port = 5432

In [2]:
# database.close()

In [3]:
from sqlalchemy import create_engine

url = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(url)

 # Załadowanie ramek do obszaru roboczego
 Implementacja funkcji `load_raw_data`, parametr `file_name` - nazwa pliku do zaczytania

In [4]:
import pandas as pd
import psycopg2

def load_raw_data(file_name):
    df = pd.read_csv(file_name,encoding = 'utf-8', sep=';', decimal='.')
    df.columns = [col.lower() for col in df.columns]
    return df


 # Zaczytanie poszczególnych plików do ramek

In [5]:
aircraft_df= load_raw_data(r'..\data\raw\aircraft.csv')
airport_list_df= load_raw_data(r'..\data\raw\airport_list.csv')
airport_weather_df= load_raw_data(r'..\data\raw\airport_weather.csv')
flight_df= load_raw_data(r'..\data\raw\flight.csv')

 # Eksport danych na bazę
Implementację metody `export_table_to_db`, argumenty:
 * `df` - ramka do eksportu,
 * `table_name` - nazwa ramki na bazie.

In [6]:
def export_table_to_db(df, table_name):
    print(f"Loading data into {table_name}...")
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False, chunksize=1000)
    print(f"Data loaded into {table_name}")

In [7]:
# import psycopg2

# connection = psycopg2.connect(
#     host='localhost',
#     user='postgres',  
#     password='postgres', 
#     dbname='airlines', 
# )
# cursor = connection.cursor()

# cursor.execute("select * from aircraft limit 5")
# #     "TRUNCATE TABLE aircraft")

 ## Wgrywanie danych

In [8]:
export_table_to_db(aircraft_df, 'aircraft')

Loading data into aircraft...
Data loaded into aircraft


In [9]:
export_table_to_db(airport_weather_df, 'airport_weather')

Loading data into airport_weather...
Data loaded into airport_weather


In [10]:
export_table_to_db(flight_df, 'flight')

Loading data into flight...
Data loaded into flight


In [11]:
export_table_to_db(airport_list_df, 'airport_list')

Loading data into airport_list...
Data loaded into airport_list
